In [37]:
# !pip install -r requirements.txt

In [2]:
import pandas as pd
import geopandas as gpd

from utils import *
import intake
import gcsfs

import calitp
from calitp.tables import tbl
from siuba import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
catalog = intake.open_catalog('./catalog.yml')

# Accessible Transit Metrics

### New Accessibilty Metric (Area)

The % of non-water area of California that is within 1/4 mi of a bus stop or 1 mi of a ferry/rail stop that is explicitly wheelchair accessible (and if in a station, that station has explicit pathways coding), and that has is served by a public-funded, open to the general public transit service with GTFS Schedule data that is served by a service that is explicitly wheelchair accessible

### New Accessibility Metric (Population)

 The % of Californians that are within 1/4 mi of a bus stop or 1 mi of a ferry/rail stop that is explicitly wheelchair accessible (and if in a station, that station has explicit pathways coding), and that has is served by a public-funded, open to the general public transit service with GTFS Schedule data that is served by a service that is explicitly wheelchair accessible

### Notes and Caveats:

* The vast majority of accessible services don't appear to provide accessibility data in GTFS
* Pathways appears to be empty in data warehouse for now...

In [4]:
total_pop_var = 'B01001_001E'

In [5]:
## can hit Census API limits with frequent runs?

# blockgrp_pop = get_census_ca_counties(total_pop_var, 'block%20group')
# blockgrp_pop = blockgrp_pop.rename(columns={'B01001_001E': 'block_grp_pop',
#                                            'block group': 'block_grp'})

In [6]:
## read pre-fetched 2019 ACS data from catalog
blockgrp_pop = (catalog.ca_bg_population.read()
                .rename(columns={'block group': 'block_grp',
                                'population': 'block_grp_pop'})
                .astype({'block_grp_pop': 'int64', 'state': 'int64',
                        'county': 'int64', 'tract': 'int64',
                        'block_grp': 'int64'})
               )

In [7]:
blockgrp_pop.head(3)

,NAME,block_grp_pop,state,county,tract,block_grp
0,"Block Group 2, Census Tract 12, Merced County,...",1388,6,47,1200,2
1,"Block Group 4, Census Tract 12, Merced County,...",1531,6,47,1200,4
2,"Block Group 1, Census Tract 12, Merced County,...",1405,6,47,1200,1


In [8]:
accessible_stops_trips = get_stops_and_trips(filter_accessible = True)

In [9]:
accessible_stops_trips.head(3)

,stop_id,route_type,stop_lon,stop_lat,calitp_itp_id,calitp_url_number,wheelchair_boarding,wheelchair_accessible,geometry
0,3820402,3,-122.077278,37.394672,217,0,1,1,POINT (-183632.839 -67099.301)
1,3820733,3,-122.053216,37.395497,217,0,1,1,POINT (-181504.056 -67053.953)
2,7269843,3,-122.111591,37.404887,217,0,1,1,POINT (-186640.338 -65897.079)


In [10]:
bus_route_types = ['3', '11']

def buffer_by_route_type(row):
    '''
    Buffer bus stops by 400 meters (.25mi),
    rail/ferry by 1600 meters (1mi)
    '''
    if row.route_type in bus_route_types:
        row.geometry = row.geometry.buffer(400)
    else:
        row.geometry = row.geometry.buffer(1600)
    return row

In [11]:
accessible_stops_trips = accessible_stops_trips.apply(buffer_by_route_type, axis=1)

In [12]:
ca_block_geo = catalog.ca_block_groups.read()
ca_block_geo = ca_block_geo.to_crs('EPSG:4326')

In [13]:
stanford_shorelines = catalog.stanford_shorelines.read()

In [14]:
ca_shoreline = stanford_shorelines >> filter(_.STFIPS == '06')

In [15]:
ca_block_geo = ca_block_geo.clip(ca_shoreline)

In [16]:
ca_block_geo = ca_block_geo.to_crs('EPSG:6414')

In [17]:
## drop large block groups (not useful access data...)
## 4 sq km threshold
ca_block_geo = ca_block_geo[ca_block_geo.geometry.area < 4e+06]

In [18]:
ca_block_geo = ca_block_geo.astype({'STATEFP': 'int64',
                                            'COUNTYFP': 'int64',
                                            'TRACTCE': 'int64',
                                            'BLKGRPCE': 'int64'})
ca_block_geo = ca_block_geo.rename(columns={'STATEFP': 'state',
                                                   'COUNTYFP': 'county',
                                                   'TRACTCE': 'tract',
                                                   'BLKGRPCE': 'block_grp'})

In [19]:
geo_stops_joined = ca_block_geo.sjoin(accessible_stops_trips, how='inner', predicate='intersects')

In [20]:
geo_stops_joined = geo_stops_joined.drop_duplicates(subset=['GEOID'])

In [21]:
geo_stops_joined.head(3)

,state,county,tract,block_grp,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,geometry,index_right,stop_id,route_type,stop_lon,stop_lat,calitp_itp_id,calitp_url_number,wheelchair_boarding,wheelchair_accessible
16760,6,73,10009,1,060730100091,Block Group 1,G5030,S,1759046,528681,...,"MULTIPOLYGON (((275529.078 -603346.514, 275528...",1717,60365,3,-117.044353,32.552369,278,0,1,1
18465,6,73,10009,3,060730100093,Block Group 3,G5030,S,707892,0,...,"POLYGON ((276297.187 -601745.532, 276297.401 -...",1717,60365,3,-117.044353,32.552369,278,0,1,1
18392,6,73,10013,1,060730100131,Block Group 1,G5030,S,491924,0,...,"POLYGON ((277107.189 -601286.509, 277150.566 -...",1717,60365,3,-117.044353,32.552369,278,0,1,1


In [22]:
stops_pop_joined = geo_stops_joined >> inner_join(_, blockgrp_pop, on=['state', 'county', 'tract', 'block_grp'])

In [23]:
stops_pop_joined['block_grp_pop'].sum()

2438340

In [24]:
blockgrp_pop['block_grp_pop'].sum()

39283497

#### Current Percentage of CA population in block groups within .25mi of accessible transit

In [25]:
((stops_pop_joined['block_grp_pop'].sum() / blockgrp_pop['block_grp_pop'].sum()) * 100).round(2)

6.21

In [26]:
geo_stops_joined.geometry.area.sum() ## sq meters

980091088.8572588

In [27]:
ca_block_geo.geometry.area.sum()

14393648284.97847

#### Current Percentage of CA land area in block groups within .25mi of accessible transit

In [28]:
((geo_stops_joined.geometry.area.sum() / ca_block_geo.geometry.area.sum()) * 100).round(2)

6.81

### Map

In [29]:
# simple_map(geo_stops_joined, 'calitp_itp_id')

#### Static Map (block groups near accessible transit, Bay Area)

![bay area accessiblity](accessible.png)

# General Transit Metrics

### New General Metric (by area):

The % of non-water area of Californian that is within 1/4 mi of a bus stop or 1 mi of a ferry/rail stop that has is served by a public-funded, open to the general public transit service with GTFS Schedule data


### New General Metric (by population):

The % of Californians that live within 1/4 mi of a bus stop or 1 mi of a ferry/rail stop that has is served by a public-funded, open to the general public transit service with GTFS Schedule data


In [30]:
all_stops = get_stops_and_trips(filter_accessible = False)

In [31]:
all_stops.head(3)

,stop_id,route_type,stop_lon,stop_lat,calitp_itp_id,calitp_url_number,wheelchair_boarding,wheelchair_accessible,geometry
0,11066,3,-121.196848,38.677369,111,0,None,None,POINT (-104007.858 74135.275)
1,165,3,-118.377250,33.929644,2,0,None,None,POINT (150043.655 -452686.613)
2,257,3,-118.391970,33.845902,2,0,None,None,POINT (148840.256 -461996.158)


In [32]:
all_stops_joined = (ca_block_geo
                    .sjoin(all_stops, how='inner', predicate='intersects')
                    .drop_duplicates(subset=['GEOID'])
                   )

In [33]:
all_stops_pop_joined = all_stops_joined >> inner_join(_, blockgrp_pop, on=['state', 'county', 'tract', 'block_grp'])

#### Current Percentage of CA population in block groups near transit with GTFS Schedule data

In [36]:
((all_stops_pop_joined['block_grp_pop'].sum() / blockgrp_pop['block_grp_pop'].sum()) * 100).round(2)

60.69

#### Current Percentage of CA land area in block groups near transit with GTFS Schedule data

In [35]:
((all_stops_joined.geometry.area.sum() / ca_block_geo.geometry.area.sum()) * 100).round(2)

65.29